<a href="https://colab.research.google.com/github/Debayan2004/TechnoHacksOfficial/blob/main/SocialMediaSentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!kaggle datasets download -d crowdflower/twitter-airline-sentiment

Dataset URL: https://www.kaggle.com/datasets/crowdflower/twitter-airline-sentiment
License(s): CC-BY-NC-SA-4.0
  0% 0.00/2.55M [00:00<?, ?B/s]
100% 2.55M/2.55M [00:00<00:00, 119MB/s]


In [2]:
import zipfile
import os

def unzip_file(zip_path, extract_to):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

zip_path = '/content/twitter-airline-sentiment.zip'
extract_to = '/content/'
unzip_file(zip_path, extract_to)


In [3]:
import pandas as pd
import numpy as np
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score

In [4]:
tweets_df = pd.read_csv('/content/Tweets.csv')


In [5]:

tweets_df['negativereason'] = tweets_df['negativereason'].fillna('None')
tweets_df['negativereason_confidence'] = tweets_df['negativereason_confidence'].fillna(0)
tweets_df['tweet_location'] = tweets_df['tweet_location'].fillna('Unknown')
tweets_df['tweet_coord'] = tweets_df['tweet_coord'].fillna('Unknown')

In [6]:

def clean_text(text):
    text = text.lower()  # lowercase text
    text = re.sub(f"[{string.punctuation}]", "", text)  # remove punctuation
    text = re.sub(r'\d+', '', text)  # remove numbers
    text = re.sub(r'\s+', ' ', text).strip()  # remove extra spaces
    return text

tweets_df['cleaned_text'] = tweets_df['text'].apply(clean_text)

In [7]:

X = tweets_df['cleaned_text']
y = tweets_df['airline_sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [8]:
tweets_df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone,cleaned_text
0,570306133677760513,neutral,1.0000,None,0.0000,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,Unknown,2015-02-24 11:35:52 -0800,Unknown,Eastern Time (US & Canada),virginamerica what dhepburn said
1,570301130888122368,positive,0.3486,None,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,Unknown,2015-02-24 11:15:59 -0800,Unknown,Pacific Time (US & Canada),virginamerica plus youve added commercials to ...
2,570301083672813571,neutral,0.6837,None,0.0000,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,Unknown,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada),virginamerica i didnt today must mean i need t...
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,Unknown,2015-02-24 11:15:36 -0800,Unknown,Pacific Time (US & Canada),virginamerica its really aggressive to blast o...
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,Unknown,2015-02-24 11:14:45 -0800,Unknown,Pacific Time (US & Canada),virginamerica and its a really big bad thing a...


In [10]:
tweets_df.describe()

,tweet_id,airline_sentiment_confidence,negativereason_confidence,retweet_count
count,1.464000e+04,14640.000000,14640.000000,14640.000000
mean,5.692184e+17,0.900169,0.458755,0.082650
std,7.791112e+14,0.162830,0.401057,0.745778
min,5.675883e+17,0.335000,0.000000,0.000000
25%,5.685592e+17,0.692300,0.000000,0.000000
50%,5.694779e+17,1.000000,0.624600,0.000000
75%,5.698905e+17,1.000000,0.703300,0.000000
max,5.703106e+17,1.000000,1.000000,44.000000


In [11]:

vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [12]:

model = MultinomialNB()
model.fit(X_train_tfidf, y_train)

MultinomialNB()

In [13]:
# Predict on the test set
y_pred = model.predict(X_test_tfidf)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)


Accuracy: 0.6881830601092896
Classification Report:
               precision    recall  f1-score   support

    negative       0.68      1.00      0.81      1889
     neutral       0.78      0.12      0.20       580
    positive       0.96      0.14      0.25       459

    accuracy                           0.69      2928
   macro avg       0.80      0.42      0.42      2928
weighted avg       0.74      0.69      0.60      2928



In [14]:
predictions = pd.DataFrame({'tweet_id': tweets_df.loc[X_test.index, 'tweet_id'], 'predicted_sentiment': y_pred})
print(predictions.head())

                 tweet_id predicted_sentiment
4794   569731104070115329            negative
10480  569263373092823040            negative
8067   568818669024907264            negative
8880   567775864679456768            negative
8292   568526521910079488            negative
